---
---
Project Notebook

Author: Brenden Eum (2024)

This is the main notebook for running all code related to the project.

---
---

# Preamble (+Select Dataset)

Set the dataset you want to work with here! (*In analysis/helpers/utilities/currentDataset.R, you can set which dataset you want to work with. Options are 'exploratory', 'confirmatory', or 'joint'. This will run all the code using THAT particular dataset.*)

In [32]:
cd("/Users/brenden/Desktop/ddm-sampling-weights")
import Pkg;
Pkg.add("RCall");
using RCall;

write("analysis/helpers/utilities/currentDataset.R", "dataset <- 'exploratory'");

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


---

# Preprocessing

**CLEAN:**

Generate variables and transform the data into something that looks like the final dataset.

**FILTER:**

Subjects are run through 1 trial-level filter and 3 participant-level filters.

At the *trial-level*, we drop any trial that is faster than .325 seconds. We selected this cutoff based on the average RT of the worst-performing participant under time pressure from Milosavljevic et al. (2010), who had an accuracy rate of 63.5%.

At the *participant-level*, we drop any participants who:
1. [Missing] Are missing data on more than 95% of trials. Missing can occur if participant [1] took too long (>12s) or [2] decided too quickly (<.325s).
2. [Accuracy] Do not perform better than chance on the trials that they did respond.
3. [Speed] Answered too quickly on average. Their average RT must be longer than .350 seconds, which is the average of the mean non-decision time estimated in Milosavljevic et al. (2010).

**SPLIT:**

Split the data in exploratory, confirmatory, and joint datasets.

| IN | OUT |
| --- | --- |
| data/raw_data/* | data/processed_data/* |
| | analysis/output/text/preprocessing/* |

In [34]:
R"source('analysis/preprocessing/clean_data.R')";
R"source('analysis/preprocessing/filter_data.R')";
R"source('analysis/preprocessing/split_data.R')";
R"source('analysis/preprocessing/preprocessing_analysis.R')";

[1] "[NAs introduced by coercion] error is ok!"


**NOTES**

59 subjects collected. 52 subjects passed the filters. 26 subjects to exploratory, 26 to confirmatory. Trials dropped $\in [0,14]$, with $mean=.96$.

| Variable | Description|
| --- | ---|
| parcode | Participant number. |
| trial | Trial number. |
| choice | 1=play, 0=skip the slot machine on this trial. |
| rt | Response time in seconds. |
| correct | 1 if participant \{played and slot machine mean was >= 0\} or \{skipped and slot machine mean was <= 0\}. 0 otherwise. |
| slot_mean | Mean of slot machine distribution. N(slot_mean, slot_sd). |
| slot_sd | St. Dev. of slot machine distribution. N(slot_mean, slot_sd). |
| sample_bin | A sample from the slot machine, binned at {-2.5, -1.5, ..., 1.5, 2.5} |
| sample_num | Which sample was this in the trial? |
| fix_dur | How long was the sample displayed for? |
| firstSample | Was this the first sample in the trial? |
| middleSample | Was this a middle sample in the trial? |
| lastSample | Was this the last sample of the trial? |
| IDnumber | The private ID# assigned to the participant by Gorilla (not identifiable outside project repo) |
| sample | What was the sample displayed? (This is binned too, since samples were converted to a finite set of stimuli) |
| raw_sample | The raw sample that was drawn from N(slot_mean, slot_sd). This is later binned to convert to dot size. |
| sample_vector | Each trial can have a maximum of 40 predetermined samples. This is the vector of potential samples. |
| dataset | Exploratory, confirmatory, or joint dataset? See Inference Strategy in Methods for details. |

---
---
# Model-Free Analyses
---
---

---
## Basic Psychometrics

Choice as a function of slot machine mean. Response time as a function of slot machine mean. Number of samples as a function of slot machine mean (this should look a lot like RT).

| IN | OUT |
| --- | --- |
| data/processed_data/* | analysis/output/figures/BasicPsychometrics_* |

In [28]:
R"source('analysis/helpers/model_free_analysis/BasicPsychometrics_Choice.R')";
R"source('analysis/helpers/model_free_analysis/BasicPsychometrics_RT.R')";
R"source('analysis/helpers/model_free_analysis/BasicPsychometrics_NumSamples.R')";

**NOTES**

Psychometric curve shows evidence of loss aversion. When slot machine mean is 0, people select skip more than they select play. Response times (and therefore number of samples) is highest when slot machine mean is close to 0 (i.e. when it's the most difficult).

---
## Choice Process

How is the choice process affected by features of the experiment?

First I plot RT as a function of the first sample. I truncate first sample to [-7, 7] since there's hardly any data outside these bounds (makes sense since most extreme slot machines are $\pm 2$ with $SD=2$.)

Next, I plot the total proportion of trials where the participant chooses (or has chosen) to play, skip, or remain undecided over the course of an entire decision, averaged over all participants. This helps us check if behavior is consistent with a sequential sampling model.

| IN | OUT |
| --- | --- |
| data/processed_data/* | analysis/output/figures/ChoiceProcess_* |

In [29]:
R"source('analysis/helpers/model_free_analysis/ChoiceProcess_RT_FirstSample.R')";

R"source('analysis/helpers/model_free_analysis/ChoiceProcess_DataTransforming.R')";
R"source('analysis/helpers/model_free_analysis/ChoiceProcess_PrPlay_Time_AvgSample.R')";
R"source('analysis/helpers/model_free_analysis/ChoiceProcess_PrSkip_Time_AvgSample.R')";
R"source('analysis/helpers/model_free_analysis/ChoiceProcess_PrUnd_Time_AvgSample.R')";

**NOTES**

Participants respond more quickly as the first sample becomes more extreme. This is predicted by sequential sampling.

The probability of playing increases over time and level shifts upward as average sample increases. The probability of skipping increases over time and level shifts upward as average sample decreases. The probability of remaining undecided decreases over the course of a trial, faster for very large, positive samples.

---
## Choice Biases

Plot psychometric curves, with a different line for each level of sample. Samples are binned with widths of 2: [-Inf, -2), [-2, -1), [-1, 0), [0, 1), [1,2), [2, Inf].

We look at the influence of the first sample, middle samples, the last sample (which is typically only viewed for a blink of an eye due to the stimulus refresh rate), early samples (2, 3, 4), and late samples (-4, -3, -2). Note that when RTs are less than 1.8 seconds, these early and late samples can overlap. For instance, if RT is 1.8 s, early sample 4 might be the same as late sample -4.

This analysis shows us how order and extremity effects interact with each other during the choice process.

| IN | OUT |
| --- | --- |
| data/processed_data/* | analysis/output/figures/ChoiceBiases_* |

In [30]:
#R"source('analysis/helpers/model_free_analysis/ChoiceBiases_First.R')"
#R"source('analysis/helpers/model_free_analysis/ChoiceBiases_Middle.R')"
#R"source('analysis/helpers/model_free_analysis/ChoiceBiases_Last.R')"
#R"source('analysis/helpers/model_free_analysis/ChoiceBiases_Early.R')"
#R"source('analysis/helpers/model_free_analysis/ChoiceBiases_Late.R')"

R"source('analysis/helpers/model_free_analysis/ChoiceBiases_Samples1-10.R')";
R"source('analysis/helpers/model_free_analysis/ChoiceBiases_Samples-10--1.R')";

**NOTES**

There is noticeable separation in the lines for all but last samples, suggesting that participants are integrating this evidence. Separation in first sample is larger than for middle samples, suggesting that the first sample might be weighted more than following samples. However, when we look at samples 2, 3, and 4, the separation is smaller than for samples -4, -3, -2 (indexed from the last sample, -1). I think this would mean that late samples are weighted more than early samples.

---
---
# Model Analysis

---
---

## Prepare data for model analysis

The DDM Toolbox we are using requires two files: expdata.csv and fixations.csv. Expdata contains trial-level data, and fixations constains fixation data for each trial. Our main dataset was created in a way to make conversion to these toolbox files easy.

| IN | OUT |
| --- | --- |
| data/processed_data/* | data/processed_data/*/expdata.csv |
|  | data/processed_data/*/fixations.csv |

In [ ]:
R"source('analysis/helpers/parameter_recovery/convert_data_for_sims.R')";

## Parameter Recovery

We want to test:
- Is drift constant?
- Does noise scale with value?

This requires more parameters in the DDM. We need to make sure that the DDM is recoverable with these new parameters.

In [37]:
# julia --project=/Users/brenden/Desktop/ddm-sampling-weights/DDMToolbox.jl --threads=4
# include("analysis/helpers/parameter_recovery/ddm_parameter_recovery.jl")

---
## How does loss aversion affect the DDM?

Estimate loss aversion using a logistic regression with total gains and losses, or with the point of indifference in a logistic regression with just slot_mean.

In [ ]:
R"source('analysis/helpers/loss_aversion/estimate_loss_aversion.R')";